#SYSC 5906:
## **Room Detection - Dataset Processor: PKL**
## Verison 2
---
Script to process a subset of the *cleaned* MIT Indoor Scenes dataset. This generates a new dataset to train a room/environment classifier, this version includes X/Y coordinates for each object and only includes a subset of the room types.

*This script is meant to run on the modified/condensed version of the MIT
dataset.*

This script **outputs the data as a .pkl file**, for use with Keras/TF.

###Step 1: Access Drive
Mount the drive with the provided .zip file of code located in it

In [ ]:
#Enter the gdrive
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
#Install missing libraries
%%capture 
!pip install pandas_read_xml

##Step 2: Setup
Import relevenat libraries, load original MIT dataset into colabs (without the images) and get the folders inside

In [ ]:
import pandas as pd
import xml.etree.ElementTree as et
import os
import pickle
from pathlib import Path
import glob
import csv
import numpy as np
from collections import Counter
import pandas_read_xml as pdx
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables

CLEANED_DATASET_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/datasets/mit_indoors/CLEANED_indoorCVPR_09annotations/CleanAnnotations/'
PICKLE_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/datasets/mit_indoors/processed/data_subset/7 classes/'

In [ ]:
#List of desired rooms to create a subset of the MIT dataset
#desiredRooms = ['bathroom','bedroom','children_room','dining_room','corridor','garage','livingroom','kitchen','office','pantry','computerroom','staircase','closet']
desiredRooms = ['bathroom','bedroom','dining_room','corridor','livingroom','kitchen','office']


In [ ]:
#Empty list for room files (which are condsensed MIT folders)
fileList = []

#Read XML's from dataset folder
with os.scandir(CLEANED_DATASET_DIRECTORY) as entries:
    for entry in entries:
        #Keep only a subset of the rooms/files in the dataset
        for room in desiredRooms:
            entryStr = str(entry.name)
            if entryStr.startswith(room):
                #Extract file plus ground truth (room types)
                fileList.append(tuple((entry.name,room)))
                #print(entry.name)

##Step 3: Parse XML
Extract object data from .xml's in the modified MIT dataset

In [ ]:
#Variables to change the end scale size of the images
scaled_height = 50
scaled_width = 50

#List for each instance of a object in the dataset
listOfAllObj = []
listOfAllFrames=[]
numFiles = 0

#List of all unique instances of a object
uniqueObjs = set()
objectMap = []

#Cycle through all collected files
for file in fileList:
    #Open relevant files
    if file[0].endswith("xml"): 
        numFiles += 1
        file_frame=np.zeros((scaled_height,scaled_width))

        #Create a list to hold objects temporarily
        fileObj = []

        #Add room label integer to instance
        fileObj.append(desiredRooms.index(file[1]))

        #Open the xml
        curXML = open(CLEANED_DATASET_DIRECTORY+"/"+file[0],"r",
                        encoding="UTF-8",errors='xmlcharrefreplace')
        fileContents = curXML.read()
        
        #Create parser to run through the xml
        eTreeParser = et.XMLParser()
        
        #Create a element tree from the xml
        etree = et.fromstring(fileContents, parser=eTreeParser)

        #Get width and height of original image
        width = etree.findall("width")[0].text
        height = etree.findall("height")[0].text

        #Run through all of the object tags inside the element tree
        curObjs = etree.findall(".//object")
        for obj in curObjs:
            #Extract X-Y coordinates for each object
            objStr = obj.findall("name")[0].text
            pts = obj.findall("polygon")[0].findall("pt")
            x_min = 2000
            y_min = 2000
            x_max = 0
            y_max = 0
            for pt in pts:
                x_val = int(pt.findall("x")[0].text)
                y_val = int(pt.findall("y")[0].text)
                x_max = max(x_max,x_val)
                x_min = min(x_min,x_val)
                y_max = max(y_max,y_val)
                y_min = min(y_min,y_val)
            #Calculate centroid of current obj
            y_avg = (y_max - y_min)/2
            x_avg = (x_max - x_min)/2
            cleanObjStr = objStr.strip('\n')

            #Scale the centroids to a consistent size
            y_avg_scaled = round(y_avg/int(height)*scaled_height)
            x_avg_scaled = round(x_avg/int(width)*scaled_width)

            # print(""+str(x_avg) + "/" + width + " -> " + str(x_avg_scaled)+"/"+str(scaled_width))
            # print(""+str(y_avg) + "/" + height+" -> " + str(y_avg_scaled)+"/"+str(scaled_height))

            #Replace object string with integer
            if cleanObjStr not in uniqueObjs:
                objectMap.append(cleanObjStr)
            
            # while(1):
            #     if file_frame[y_avg_scaled][x_avg_scaled] == 0:
            #         file_frame[y_avg_scaled][x_avg_scaled] = objectMap.index(cleanObjStr)
            #         break
            #     else:
            #         x_avg_scaled = x_avg_scaled + 1
            #         if(x_avg_scaled > scaled_width-1):
            #             x_avg_scaled = x_avg_scaled -1
            #             y_avg_scaled = y_avg_scaled + 1
                    
            #Store object name and position (centroid)
            file_frame[y_avg_scaled,x_avg_scaled] = objectMap.index(cleanObjStr)

            obj_tuple = [objectMap.index(cleanObjStr),x_avg,y_avg]

            fileObj.append(obj_tuple)
            
            uniqueObjs.add(cleanObjStr)
            
    #Store list of all objects from the current file
    listOfAllFrames.append((file_frame,desiredRooms.index(file[1])))
    listOfAllObj.append(fileObj)
print(listOfAllFrames[0])

(array([[ 0., 14.,  0., ...,  0.,  0.,  0.],
       [17.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 15., 11., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), 3)


##Step 4: Pickle the data
Process extracted data into a Pickle to be used with Keras

The format looks like:\
[room label [obj1 tuple] [obj1 tuple] ... [objn tuple]

Where each tuple has: Obj label, x_avg, y_avg

In [ ]:
#Pickle the data so we can access it later
pckl = open(PICKLE_DIRECTORY+"listOfAllObjLoc_v2.pkl","wb")
pickle.dump(listOfAllFrames,pckl)
pckl.close()

#Test pickle
# new_pckl = open(PICKLE_DIRECTORY+"listOfAllObjLoc.pkl","rb")
# new_list = pickle.load(new_pckl)
# new_pckl.close()